In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import time

In [36]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

In [37]:
ds1 = xr.open_dataset(input_folder+'765041.T2.wrf2d_d01_1996-04-28_15:00:00.nc')
ds2 = xr.open_dataset(input_folder+'765041.TD2.wrf2d_d01_1996-04-28_15:00:00.nc')

In [38]:
ds3 = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2020-12-05_03:00:00.nc')

In [40]:
start_year = 1979
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.T2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
        # 提取CONUS数据
            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['T2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['T2'].dims[1:], coords={'south_north': ds['T2'].coords['south_north'], 'west_east': ds['T2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['T2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'t2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'T2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-02 20:12:48
2024-11-02 20:17:44
2024-11-02 20:22:36
2024-11-02 20:27:33
2024-11-02 20:32:43
2024-11-02 20:37:23
2024-11-02 20:45:09
2024-11-02 20:59:36
2024-11-02 21:13:55
2024-11-02 21:27:32
2024-11-02 21:42:24
2024-11-02 21:55:48
2024-11-02 22:09:37
2024-11-02 22:23:47
2024-11-02 22:37:42
2024-11-02 22:52:48
2024-11-02 23:07:06
2024-11-02 23:19:47
2024-11-02 23:34:11
2024-11-02 23:47:16
2024-11-03 00:01:15
2024-11-03 00:14:25
2024-11-03 00:27:22
2024-11-03 00:41:23


In [41]:
gdf = gpd.read_file('../../tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/CONUS404_T_dT/TarFiles/'

start_year = 1979
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)

    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:

            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  
            else:
                num_days = 28  

        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'765041.TD2.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)

            lon = ds['XLONG'].values
            lat = ds['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')
            
            mask = np.full(ds['TD2'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds['TD2'].dims[1:], coords={'south_north': ds['TD2'].coords['south_north'], 'west_east': ds['TD2'].coords['west_east']})
            ds_s = ds.where(mask_da, drop=True)
            lonn = np.linspace(-124.848, -66.885, 1137)
            latt = np.linspace(24.396, 49.384, 708)
            prec = ds_s['TD2'].values
            lat_min = latt.min()
            lat_max = latt.max()
            ds_sss = xr.Dataset({'td2': (['time', 'lat', 'lon'], prec)},
                                coords={'lon': (['lon'], lonn),
                                        'lat': (['lat'], latt),
                                        'time': ('time', ds_s.Time.values)})

            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.lat.min().values
                    lat_max = ds_lon_subset.lat.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '../CONUS404_data/LST/UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'TD2.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2024-11-03 00:55:10
2024-11-03 01:08:39
2024-11-03 01:22:23
2024-11-03 01:36:11
2024-11-03 01:50:28
2024-11-03 01:04:06
2024-11-03 01:18:12
2024-11-03 01:32:25
2024-11-03 01:47:47
2024-11-03 02:01:38
2024-11-03 02:16:55
2024-11-03 02:30:10
2024-11-03 02:42:52
2024-11-03 02:56:59
2024-11-03 03:11:16
2024-11-03 03:25:22
2024-11-03 03:39:13
2024-11-03 03:51:09
2024-11-03 04:04:51
2024-11-03 04:17:56
2024-11-03 04:32:10
2024-11-03 04:45:23
2024-11-03 04:58:17
2024-11-03 05:12:03


In [6]:
dsss = xr.open_dataset('/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/U-50/T2.wrf2d_d01_2001-03-31.nc')